# CP321 Final Project

In [142]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [143]:
datafile = "/Users/jashan/Desktop/CP321-FinalProject/data/finalData.csv"

In [144]:
raw_data = pd.read_csv(datafile)
raw_data

,REF_DATE,GEO,DGUID,Age (2),"Highest certificate, diploma or degree (16)",Major field of study - Classification of Instructional Programs (CIP) 2021 (500),Occupation - Unit group - National Occupational Classification (NOC) 2021 (821A),Gender (3),UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2021,Newfoundland and Labrador,2021A000210,25 to 64 years,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",All occupations,Men+,NaN,0,units,0,NaN,2.2.2.2.3.2,12025,NaN,NaN,NaN,0
1,2021,Newfoundland and Labrador,2021A000210,25 to 64 years,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",All occupations,Women+,NaN,0,units,0,NaN,2.2.2.2.3.3,7755,NaN,NaN,NaN,0
2,2021,Newfoundland and Labrador,2021A000210,25 to 64 years,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",0 Legislative and senior management occupations,Women+,NaN,0,units,0,NaN,2.2.2.2.4.3,10,NaN,NaN,NaN,0
3,2021,Newfoundland and Labrador,2021A000210,25 to 64 years,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",00 Legislative and senior managers,Women+,NaN,0,units,0,NaN,2.2.2.2.5.3,10,NaN,NaN,NaN,0
4,2021,Newfoundland and Labrador,2021A000210,25 to 64 years,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",000 Legislative and senior managers,Women+,NaN,0,units,0,NaN,2.2.2.2.6.3,10,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130827,2021,Nunavut,2021A000262,25 to 64 years,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",9 Occupations in manufacturing and utilities,Men+,NaN,0,units,0,NaN,14.2.4.476.725.2,10,NaN,NaN,NaN,0
130828,2021,Nunavut,2021A000262,25 to 64 years,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",90 Middle management occupations in manufactur...,Men+,NaN,0,units,0,NaN,14.2.4.476.726.2,10,NaN,NaN,NaN,0
130829,2021,Nunavut,2021A000262,25 to 64 years,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",900 Middle management occupations in manufactu...,Men+,NaN,0,units,0,NaN,14.2.4.476.727.2,10,NaN,NaN,NaN,0
130830,2021,Nunavut,2021A000262,25 to 64 years,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",9001 Managers in manufacturing and utilities,Men+,NaN,0,units,0,NaN,14.2.4.476.728.2,10,NaN,NaN,NaN,0


In [145]:
redundant_cols = [
    "DGUID",
    "Age (2)",
    "UOM",
    "UOM_ID",
    "SCALAR_FACTOR",
    "SCALAR_ID",
    "STATUS",
    "SYMBOL",
    "TERMINATED",
    "DECIMALS",
    "VECTOR",
    "REF_DATE",
    "COORDINATE",
]
master_df = raw_data.drop(columns=redundant_cols)
master_df.columns = ["GEO","EDUCATION","MAJOR","OCCUPATION","GENDER","COUNT"]
master_df

,GEO,EDUCATION,MAJOR,OCCUPATION,GENDER,COUNT
0,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",All occupations,Men+,12025
1,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",All occupations,Women+,7755
2,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",0 Legislative and senior management occupations,Women+,10
3,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",00 Legislative and senior managers,Women+,10
4,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",000 Legislative and senior managers,Women+,10
...,...,...,...,...,...,...
130827,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",9 Occupations in manufacturing and utilities,Men+,10
130828,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",90 Middle management occupations in manufactur...,Men+,10
130829,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",900 Middle management occupations in manufactu...,Men+,10
130830,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",9001 Managers in manufacturing and utilities,Men+,10


In [146]:
master_df.isnull().sum()

GEO           0
EDUCATION     0
MAJOR         0
OCCUPATION    0
GENDER        0
COUNT         0
dtype: int64

Fix Gender

In [147]:
def clean_gender_COUNTs(COUNT):
    return COUNT[:-1]


def get_noc_codes(COUNT):
    noc = COUNT.split(" ")[0]
    if not noc.isdigit():
        return np.nan
    return noc

def get_hierarchy_number(COUNT):
    if type(COUNT) is float:
        return 0
    return len(COUNT)

def clean_occupations(COUNT):
    COUNT = COUNT.split(" ")
    COUNT.pop(0)
    return " ".join(COUNT)


master_df["GENDER"] = master_df.GENDER.apply(clean_gender_COUNTs) # remove plus sign form gender COUNTs
master_df["NOC"] = master_df.OCCUPATION.apply(get_noc_codes) # Split NOC codes based on categories.
master_df["Hierarchy"] = master_df.NOC.apply(get_hierarchy_number)
master_df["OCCUPATION_C"] = master_df.OCCUPATION.apply(clean_occupations)

master_df

,GEO,EDUCATION,MAJOR,OCCUPATION,GENDER,COUNT,NOC,Hierarchy,OCCUPATION_C
0,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",All occupations,Men,12025,NaN,0,occupations
1,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",All occupations,Women,7755,NaN,0,occupations
2,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",0 Legislative and senior management occupations,Women,10,0,1,Legislative and senior management occupations
3,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",00 Legislative and senior managers,Women,10,00,2,Legislative and senior managers
4,Newfoundland and Labrador,"No certificate, diploma or degree","No postsecondary certificate, diploma or degree",000 Legislative and senior managers,Women,10,000,3,Legislative and senior managers
...,...,...,...,...,...,...,...,...,...
130827,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",9 Occupations in manufacturing and utilities,Men,10,9,1,Occupations in manufacturing and utilities
130828,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",90 Middle management occupations in manufactur...,Men,10,90,2,Middle management occupations in manufacturing...
130829,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",900 Middle management occupations in manufactu...,Men,10,900,3,Middle management occupations in manufacturing...
130830,Nunavut,"Postsecondary certificate, diploma or degree","Personal, protective and transportation services",9001 Managers in manufacturing and utilities,Men,10,9001,4,Managers in manufacturing and utilities


In [148]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130832 entries, 0 to 130831
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   GEO           130832 non-null  object
 1   EDUCATION     130832 non-null  object
 2   MAJOR         130832 non-null  object
 3   OCCUPATION    130832 non-null  object
 4   GENDER        130832 non-null  object
 5   COUNT         130832 non-null  int64 
 6   NOC           130477 non-null  object
 7   Hierarchy     130832 non-null  int64 
 8   OCCUPATION_C  130832 non-null  object
dtypes: int64(2), object(7)
memory usage: 9.0+ MB


In [149]:
master_df.GEO.unique()

array(['Newfoundland and Labrador', 'Prince Edward Island', 'Nova Scotia',
       'New Brunswick', 'Quebec', 'Ontario', 'Manitoba', 'Saskatchewan',
       'Alberta', 'British Columbia', 'Yukon', 'Northwest Territories',
       'Nunavut'], dtype=object)

In [150]:
broader_cats = master_df[master_df.Hierarchy==1].OCCUPATION_C.unique()
broader_cats

array(['Legislative and senior management occupations',
       'Business, finance and administration occupations',
       'Natural and applied sciences and related occupations',
       'Health occupations',
       'Occupations in education, law and social, community and government services',
       'Occupations in art, culture, recreation and sport',
       'Sales and service occupations',
       'Trades, transport and equipment operators and related occupations',
       'Natural resources, agriculture and related production occupations',
       'Occupations in manufacturing and utilities'], dtype=object)

In [151]:
temp_df = master_df[master_df.Hierarchy == 1].groupby(["GEO","OCCUPATION_C","GENDER"],as_index=False).COUNT.sum()
temp_df

,GEO,OCCUPATION_C,GENDER,COUNT
0,Alberta,"Business, finance and administration occupations",Men,98320
1,Alberta,"Business, finance and administration occupations",Women,248605
2,Alberta,Health occupations,Men,31680
3,Alberta,Health occupations,Women,140075
4,Alberta,Legislative and senior management occupations,Men,15080
...,...,...,...,...
255,Yukon,Occupations in manufacturing and utilities,Women,40
256,Yukon,Sales and service occupations,Men,1615
257,Yukon,Sales and service occupations,Women,1880
258,Yukon,"Trades, transport and equipment operators and ...",Men,3435


In [152]:
alberta = temp_df[temp_df.GEO == "Alberta"].copy()
alberta

,GEO,OCCUPATION_C,GENDER,COUNT
0,Alberta,"Business, finance and administration occupations",Men,98320
1,Alberta,"Business, finance and administration occupations",Women,248605
2,Alberta,Health occupations,Men,31680
3,Alberta,Health occupations,Women,140075
4,Alberta,Legislative and senior management occupations,Men,15080
5,Alberta,Legislative and senior management occupations,Women,6370
6,Alberta,Natural and applied sciences and related occup...,Men,132840
7,Alberta,Natural and applied sciences and related occup...,Women,41165
8,Alberta,"Natural resources, agriculture and related pro...",Men,57725
9,Alberta,"Natural resources, agriculture and related pro...",Women,15175


In [153]:
fig = px.bar(
    alberta,
    x="OCCUPATION_C",
    y="COUNT",
    color="GENDER",
    barmode="group",
    title="Gender Distribution Across Occupations in Alberta",
    labels={"OCCUPATION_C": "Occupation Code", "Count": "Number of People"},
)

fig.update_layout(xaxis={"categoryorder": "total descending"})
fig.update_layout(
    xaxis_tickangle=-45,
    plot_bgcolor="white",
    hovermode="x unified",
    legend_title_text="Gender",
)

fig.update_traces(
    textfont_size=12, textangle=0, textposition="outside", cliponaxis=False
)

fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'cliponaxis': False,
              'hovertemplate': 'GENDER=Men<br>Occupation Code=%{x}<br>COUNT=%{y}<extra></extra>',
              'legendgroup': 'Men',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'Men',
              'offsetgroup': 'Men',
              'orientation': 'v',
              'showlegend': True,
              'textangle': 0,
              'textfont': {'size': 12},
              'textposition': 'outside',
              'type': 'bar',
              'x': array(['Business, finance and administration occupations',
                          'Health occupations', 'Legislative and senior management occupations',
                          'Natural and applied sciences and related occupations',
                          'Natural resources, agriculture and related production occupations',
                          'Occupations in art, culture, recreation and sport',
                          'Occupations in education, law and social, community and government services',
                          'Occupations in manufacturing and utilities',
                          'Sales and service occupations',
                          'Trades, transport and equipment operators and related occupations'],
                         dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'EIABAMB7AADoOgAA6AYCAH3hAAB0RQAA4CgBAFnsAACQmgIAc30FAA==', 'dtype': 'i4'},
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'cliponaxis': False,
              'hovertemplate': 'GENDER=Women<br>Occupation Code=%{x}<br>COUNT=%{y}<extra></extra>',
              'legendgroup': 'Women',
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': 'Women',
              'offsetgroup': 'Women',
              'orientation': 'v',
              'showlegend': True,
              'textangle': 0,
              'textfont': {'size': 12},
              'textposition': 'outside',
              'type': 'bar',
              'x': array(['Business, finance and administration occupations',
                          'Health occupations', 'Legislative and senior management occupations',
                          'Natural and applied sciences and related occupations',
                          'Natural resources, agriculture and related production occupations',
                          'Occupations in art, culture, recreation and sport',
                          'Occupations in education, law and social, community and government services',
                          'Occupations in manufacturing and utilities',
                          'Sales and service occupations',
                          'Trades, transport and equipment operators and related occupations'],
                         dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'HcsDACsjAgDiGAAAzaAAAEc7AADbagAAUWYCAIk/AAD2sAMARpsAAA==', 'dtype': 'i4'},
              'yaxis': 'y'}],
    'layout': {'barmode': 'group',
               'hovermode': 'x unified',
               'legend': {'title': {'text': 'Gender'}, 'tracegroupgap': 0},
               'plot_bgcolor': 'white',
               'template': '...',
               'title': {'text': 'Gender Distribution Across Occupations in Alberta'},
               'xaxis': {'anchor': 'y',
                         'categoryorder': 'total descending',
                         'domain': [0.0, 1.0],
                         'tickangle': -45,
                         'title': {'text': 'Occupation Code'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'COUNT'}}}
})

In [154]:
industry = temp_df[
    temp_df.OCCUPATION_C == "Business, finance and administration occupations"
].copy()
industry

,GEO,OCCUPATION_C,GENDER,COUNT
0,Alberta,"Business, finance and administration occupations",Men,98320
1,Alberta,"Business, finance and administration occupations",Women,248605
20,British Columbia,"Business, finance and administration occupations",Men,121755
21,British Columbia,"Business, finance and administration occupations",Women,287075
40,Manitoba,"Business, finance and administration occupations",Men,28670
41,Manitoba,"Business, finance and administration occupations",Women,67945
60,New Brunswick,"Business, finance and administration occupations",Men,14880
61,New Brunswick,"Business, finance and administration occupations",Women,39110
80,Newfoundland and Labrador,"Business, finance and administration occupations",Men,7625
81,Newfoundland and Labrador,"Business, finance and administration occupations",Women,22885


In [167]:
fig = px.bar(
    industry,
    x="GEO",
    y="COUNT",
    color="GENDER",
    barmode="group",
    log_y=True,
    template="plotly_white",
    hover_name="GEO",
    hover_data={"GENDER":False,"GEO":False},
)

fig.update_layout(xaxis={"categoryorder": "total descending"})

fig.update_layout(
    xaxis_tickangle=-30,
    xaxis=dict(
        categoryorder="total descending",
        tickangle=-30,
        title="<b>Administrative Units</b>",
        tickfont=dict(size=12),
        title_font=dict(size=12 ),
    ),
    yaxis=dict(
        title="<b>Number of People (Log Scale)</b>",
        tickfont=dict(size=12),
        title_font=dict(size=12),
        showgrid=True,
        gridcolor="rgba(200, 200, 200, 0.3)",
    ),
    height=600,
    margin=dict(l=50, r=50, t=80, b=100),
)

# update Tile
fig.update_layout(
    title=dict(
        text="<b>Gender Composition of  Aerospace Engineering Departments Across Canada</b>",
        font=dict(size=20, family="Arial", color="#2a4978"),
    )
)
fig.update_layout(
    legend=dict(
        orientation="h",
        title=None,
        y=1,
        x=1,
        xanchor="right",
        yanchor="bottom",
    ),
    legend_bordercolor="black",
    legend_borderwidth=1,
)
fig.update_traces(
    textfont_size=1,
    textangle=0,
    textposition="outside",
    cliponaxis=False,
    texttemplate="%{y:.2s}", 
)

fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'cliponaxis': False,
              'customdata': array([['Men'],
                                   ['Men'],
                                   ['Men'],
                                   ['Men'],
                                   ['Men'],
                                   ['Men'],
                                   ['Men'],
                                   ['Men'],
                                   ['Men'],
                                   ['Men'],
                                   ['Men'],
                                   ['Men'],
                                   ['Men']], dtype=object),
              'hovertemplate': '<b>%{hovertext}</b><br><br>COUNT=%{y}<extra></extra>',
              'hovertext': array(['Alberta', 'British Columbia', 'Manitoba', 'New Brunswick',
                                  'Newfoundland and Labrador', 'Northwest Territories', 'Nova Scotia',
                                  'Nunavut', 'Ontario', 'Prince Edward Island', 'Quebec', 'Saskatchewan',
                                  'Yukon'], dtype=object),
              'legendgroup': 'Men',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'Men',
              'offsetgroup': 'Men',
              'orientation': 'v',
              'showlegend': True,
              'textangle': 0,
              'textfont': {'size': 1},
              'textposition': 'outside',
              'texttemplate': '%{y:.2s}',
              'type': 'bar',
              'x': array(['Alberta', 'British Columbia', 'Manitoba', 'New Brunswick',
                          'Newfoundland and Labrador', 'Northwest Territories', 'Nova Scotia',
                          'Nunavut', 'Ontario', 'Prince Edward Island', 'Quebec', 'Saskatchewan',
                          'Yukon'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'EIABAJvbAQD+bwAAIDoAAMkdAACJAwAAUUoAADACAADvCgYAlQsAAI93AwCRSwAAUgMAAA==', 'dtype': 'i4'},
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'cliponaxis': False,
              'customdata': array([['Women'],
                                   ['Women'],
                                   ['Women'],
                                   ['Women'],
                                   ['Women'],
                                   ['Women'],
                                   ['Women'],
                                   ['Women'],
                                   ['Women'],
                                   ['Women'],
                                   ['Women'],
                                   ['Women'],
                                   ['Women']], dtype=object),
              'hovertemplate': '<b>%{hovertext}</b><br><br>COUNT=%{y}<extra></extra>',
              'hovertext': array(['Alberta', 'British Columbia', 'Manitoba', 'New Brunswick',
                                  'Newfoundland and Labrador', 'Northwest Territories', 'Nova Scotia',
                                  'Nunavut', 'Ontario', 'Prince Edward Island', 'Quebec', 'Saskatchewan',
                                  'Yukon'], dtype=object),
              'legendgroup': 'Women',
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': 'Women',
              'offsetgroup': 'Women',
              'orientation': 'v',
              'showlegend': True,
              'textangle': 0,
              'textfont': {'size': 1},
              'textposition': 'outside',
              'texttemplate': '%{y:.2s}',
              'type': 'bar',
              'x': array(['Alberta', 'British Columbia', 'Manitoba', 'New Brunswick',
                          'Newfoundland and Labrador', 'Northwest Territories', 'Nova Scotia',
                          'Nunavut', 'Ontario', 'Prince Edward Island', 'Quebec', 'Saskatchewan',
                          'Yukon'], dtype=object),
              'xaxis': 'x',
              'y': {'bdat